#  Calibration Image Master Creation notebook

This notebook calibrates Bias images, Dark images and Flat images and combines them into their respective Master files

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
This project uses:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)



## How to Use

It's recommended you click "Run All Cells" to run this notebook, but you may optionally run each cell individually.

This notebook requires the location of your raw bias images as either a full location or as a relative path if you have the bias images in the folder that this notebook is located in.

Example:

Full location: C:\path\to\bias

Relative: path\to\bias



# Setup

All variables are set up in this cell. If this cell is not run, the program will not run.

In [ ]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.units import adu, second
from astropy.wcs import FITSFixedWarning
from ccdproc import ImageFileCollection, combine, subtract_bias, subtract_dark
import matplotlib.pyplot as plt
import numpy as np
import logging
import warnings

warnings.filterwarnings("ignore", category=FITSFixedWarning)


GIGABYTE = 1e9 # 1 billion bytes
MEMORY_LIMIT = 10*GIGABYTE

data_directory = Path(".") # directory that notebooks are in.
DARKS_PATH = data_directory / "Darks_20C"
BIAS_PATH = data_directory / "Bias"
FLATS_PATH = data_directory / "Flats"
REDUCED_PATH = data_directory / "reduced"
# make sure reduced path exists
REDUCED_PATH.mkdir(exist_ok=True)

#useful functions
def inv_average(data):
    return 1/np.average(data)

def inv_median(data):
    return 1/np.median(data)

def files_as_paths(directory_path):
    fits_images = ImageFileCollection(directory_path, glob_include="*.fit*")
    return list(map(lambda x: directory_path / x, fits_images.files))

def write_fits(fits_file, file_path):
        # convert to HDU so we can scale to 16 bit
        fits_file = fits_file.to_hdu(hdu_uncertainty=None)
        
  
        fits_file[0].scale("float32") 
        fits_file[0].writeto(file_path)


# Stage 1: Bias

During this stage the master Bias is made, it is integral to the rest of the process and must be done first.

In [ ]:
biases = files_as_paths(BIAS_PATH)

combined_bias = combine(biases, 
                        method="average", 
                        mem_limit = MEMORY_LIMIT,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        unit=adu)
# Annotate header with modification information
combined_bias.meta["IMAGETYP"] = "Bias"
combined_bias.meta['HISTORY'] = f"Combined {len(biases)} images by mean"

combined_bias_name = REDUCED_PATH / "combined_bias_mean.fit"

write_fits(combined_bias, combined_bias_name)


# Stage 2: Dark

Here the master Dark is made. This process requires the use of the master Bias, so Stage 1 must be run first

In [ ]:
darks = ImageFileCollection(DARKS_PATH, glob_include="*.fit*")

# Path to store bias-calibrated darks
INTERMEDIARY_PATH = Path(DARKS_PATH / "calibrated")
INTERMEDIARY_PATH.mkdir(exist_ok=True)

# combined bias is reloaded to make running all notebook cells in order is unnecessary
combined_bias_path = REDUCED_PATH / "combined_bias_mean.fit"
combined_bias = CCDData.read(combined_bias_path, unit=adu)

# for dark in darks.files:
    
#     dark_name = Path(dark).stem 
#     # add _b to denote bias subtraction
#     modified_dark_name = INTERMEDIARY_PATH / f"{dark_name}_b.fit"
    
#     # Ensure that file does not already exist
#     if not modified_dark_name.is_file():
#         # Read into memory and calibrate
#         dark_ccd = CCDData.read(DARKS_PATH / dark, unit=adu)

#         write_fits(dark_ccd, modified_dark_name, write_float=True)


combined_dark = combine(files_as_paths(DARKS_PATH), 
                        method="average", 
                        mem_limit = MEMORY_LIMIT,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        unit=adu)

combined_dark = subtract_bias(ccd=combined_dark, 
                        master=combined_bias, 
                        add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
combined_dark.data = np.around(combined_dark.data)
# Get temperature to add to name
temp = round(combined_dark.header["CCD-TEMP"])

# annotate header with important information
combined_dark.meta["IMAGETYP"] = "Dark"
combined_dark.meta['combined'] = True
combined_dark.meta['HISTORY'] = f"Combined {len(darks.files)} images by mean"
# Create new output path
combined_dark_name = REDUCED_PATH / f"combined_dark_{temp}C_mean.fit"

write_fits(combined_dark, combined_dark_name)


# Stage 3: Flat

Here the master Flat is made. This process requires the use of the master Bias and master Dark, so Stage 1 and 2 must be run first

### Part 1: Calibrate the flat files

In [ ]:
flats = ImageFileCollection(FLATS_PATH, glob_include="*.fit*")
# Path to store bias and dark-calibrated flats
INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated")
INTERMEDIARY_PATH.mkdir(exist_ok=True)

# combined bias and dark is reloaded to make running all notebook cells in order is unnecessary
combined_bias = CCDData.read(REDUCED_PATH /"combined_bias_mean.fit")
combined_dark = CCDData.read(REDUCED_PATH /"combined_dark_5C_mean.fit")

for flat in flats.files:
    # Get name and make new full filename to write to
    flat_name = Path(flat).stem 
    modified_flat_name = INTERMEDIARY_PATH / f"{flat_name}_b_d.fit"
    
    if not modified_flat_name.is_file():
        # Read into memory and calibrate
        flat_ccd = CCDData.read(FLATS_PATH / flat, unit=adu)
        flat_ccd = subtract_bias(ccd=flat_ccd, 
                                master=combined_bias, 
                                add_keyword={"HISTORY": f"Subtracted master bias combined_bias.fit"})
        flat_ccd = subtract_dark(ccd=flat_ccd, 
                                master=combined_dark, 
                                exposure_time="EXPTIME", 
                                exposure_unit=second, 
                                scale=True, # Scales exposure time to flat
                                add_keyword={"HISTORY": f"Subtracted master bias combined_dark.fit"})
        #flat_ccd = cosmicray_lacosmic(ccd=flat_ccd, gain=1.42, readnoise=15.1, fsmode="convolve", cleantype="median", gain_apply=True)
        write_fits(flat_ccd, file_path=modified_flat_name)
        


### Part 2: Combine the flat files

In [ ]:

INTERMEDIARY_PATH = Path(FLATS_PATH / "calibrated") 
paths = files_as_paths(FLATS_PATH)
    
combined_flat = combine(paths, 
                        method="median", 
                        mem_limit = MEMORY_LIMIT,
                        scale= inv_median,
                        sigma_clip=True,
                        sigma_clip_low_thresh=5,
                        sigma_clip_high_thresh=5,
                        sigma_clip_func=np.median,
                        sigma_clip_dev_func=mad_std,
                        dtype="float32",
                        unit=adu)
# Annotate header with useful information
combined_flat.meta["IMAGETYP"] = "Flat"
combined_flat.meta['combined'] = True
combined_flat.meta['HISTORY'] = f"Combined {len(paths)} images by median"
combined_flat_name = REDUCED_PATH / "combined_flat_median_calib_after.fit"

write_fits(combined_flat, file_path=combined_flat_name)

# Experimentation beyond this point

You may safely ignore everything below here.

In [ ]:
import matplotlib.pyplot as plt
from astropy.units import electron
from astropy.visualization import hist
cool_dark = CCDData.read(REDUCED_PATH / "combined_dark_1c_mean.fit", unit=adu).multiply(1.42 * electron / adu).divide(60*second)
old_dark = CCDData.read(REDUCED_PATH / "combined_dark_5c_mean.fit", unit=adu).multiply(1.42 * electron / adu).divide(90*second)

plt.figure(figsize=(20, 10))

hist(cool_dark.data.flatten(), bins=5000, density=False, label='90 sec 1c dark', alpha=0.4)
hist(old_dark.data.flatten(), bins=5000, density=False, label='90 sec 5c dark', alpha=0.4)
plt.xlabel('dark current, $e^-$/sec')
plt.ylabel('Number of pixels')
plt.loglog()
plt.legend();

In [ ]:
hot_pixels = (cool_dark.data > 3.0)
plt.figure(figsize=(10, 10))
plt.plot(cool_dark.data[hot_pixels].flatten(), old_dark.data[hot_pixels].flatten(), '.', alpha=0.2, label='Data')
plt.xlabel("dark current ($e^-$/sec), 90 sec exposure time")
plt.ylabel("dark current ($e^-$/sec), 60 sec exposure time")
plt.xlim((1, 100))
plt.ylim((1, 100))
plt.plot([0, 100], [0, 100], label='Ideal relationship')
plt.grid()
plt.legend();

In [ ]:
from astropy.units import dimensionless_unscaled
hot_pixels.sum()
as_ccd = CCDData(data=hot_pixels.astype("uint8"), unit=dimensionless_unscaled)
as_ccd.header["imagetyp"] = "dark mask"
as_ccd.write(REDUCED_PATH / "dark_current_mask.fit")

In [ ]:
from ccdproc import ccdmask

flat = REDUCED_PATH / "combined_flat.fit"
flat_ccd = CCDData.read(flat, units=adu)

flatmask = ccdmask(flat_ccd)
mask_as_ccd = CCDData(data=flatmask.astype("uint8"), unit=dimensionless_unscaled)
mask_as_ccd.header["imagetyp"] = "flat mask"
mask_as_ccd.write(REDUCED_PATH / "flat_mask.fit")


In [ ]:
combined_mask = mask_as_ccd.data | as_ccd.data
combined_mask = CCDData(data=combined_mask.astype("uint8"), unit=dimensionless_unscaled)
combined_mask.write(REDUCED_PATH / "combined_mask.fit")

In [ ]:
combined_mask.data.sum()